In [ ]:
import argparse
import gymnasium as gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Video
%matplotlib inline

In [ ]:
plt.ion()
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

# Cart Pole
gamma = 0.99
log_interval = 10

env = gym.make('CartPole-v1', render_mode="rgb_array")
state, info = env.reset()

In [ ]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [ ]:
class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, state_values

In [ ]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [ ]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()

In [ ]:
def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]

In [ ]:
episode_durations = []

def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())



In [ ]:
if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 600
else:
    num_episodes = 50
    
first_episode_images = []
last_episode_images = []

def main():
    running_reward = 10

    # run infinitely many episodes
    for i_episode in range(num_episodes):

        # reset environment and episode reward
        state, _ = env.reset()
        ep_reward = 0

        global last_episode_images
        last_episode_images = []
        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _, _ = env.step(action)

#             if args.render:
#                 env.render()

            model.rewards.append(reward)
            ep_reward += reward
            if i_episode == 0:
                first_episode_images.append(env.render())
            elif i_episode != 0:
                last_episode_images.append(env.render())

            if done:
                episode_durations.append(t+1)
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        plot_durations()
        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
#             print("Solved! Running reward is now {} and "
#                   "the last episode runs to {} time steps!".format(running_reward, t))
            break


if __name__ == '__main__':
    main()
    print('Complete')
    plot_durations(show_result=True)
    plt.ioff()
    plt.show()

In [ ]:
print("Length of first episode (in frames): ", len(first_episode_images))
print("Length of last episode (in frames): ", len(last_episode_images))

In [ ]:
def write_first_and_last_episode_gifs(first_episode_images, last_episode_images):
    import imageio
    imageio.mimwrite("./images/actor_critic_cartpole_first_episode.gif", first_episode_images, format="gif",loop=0,duration=50)
    imageio.mimwrite("./images/actor_critic_cartpole_last_episode.gif", last_episode_images, format="gif", loop=0, duration=50)

In [ ]:
write_first_and_last_episode_gifs(first_episode_images, last_episode_images)

In [ ]:
def display_episode(episode_filename):
    from IPython.display import display, Image
    with open(episode_filename, 'rb') as f:
        display(Image(f.read(), format='gif'))

In [ ]:
display_episode('./images/actor_critic_cartpole_first_episode.gif')

In [ ]:
display_episode('./images/actor_critic_cartpole_last_episode.gif')